## Compare the assemblies in GenBank, GTDB, and RAST

Each has a different set. What are the unions and intersections?

In [1]:
# A lot of this is not used, but we import it so we have it later!
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
import numpy as np

import math
import re

from PhiSpyAnalysis import theils_u, DateConverter, printmd
from PhiSpyAnalysis import read_phages, read_gtdb, read_checkv, read_base_pp, read_categories, read_metadata, read_gbk_metadata

from scipy.stats import pearsonr, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn import decomposition
from sklearn.ensemble import RandomForestClassifier

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, tukeyhsd, MultiComparison
from statsmodels.multivariate.manova import MANOVA


import subprocess
import gzip


ModuleNotFoundError: No module named 'Bio'

In [2]:
rv = re.compile('^\\w+')
def remove_ver(x):
    return rv.search(str(x)).group()

# GTDB



In [3]:
# GTDB
gtdb = read_gtdb()
gtdb['assembly_nover'] = gtdb['assembly_accession'].apply(remove_ver)
gtdb[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000012145.1,GCA_000012145
1,GCA_000013845.2,GCA_000013845
2,GCA_000016465.1,GCA_000016465
3,GCA_000024525.1,GCA_000024525
4,GCA_000152525.1,GCA_000152525
...,...,...
311475,GCA_910574395.1,GCA_910574395
311476,GCA_910589955.1,GCA_910589955
311477,GCA_910590315.1,GCA_910590315
311478,GCA_910592735.1,GCA_910592735


In [4]:
gtdb

,assembly_accession,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,...,trna_count,trna_selenocysteine_count,domain,phylum,class,order,family,genus,species,assembly_nover
0,GCA_000012145.1,GB_GCA_000012145.1,0,100.00,0.47,324,o__Rickettsiales (UID3809),211,0.0,1344326,...,33,0,Bacteria,Proteobacteria,Alphaproteobacteria,Rickettsiales,Rickettsiaceae,Rickettsia,Rickettsia felis,GCA_000012145
1,GCA_000013845.2,GB_GCA_000013845.2,0,100.00,0.00,332,o__Clostridiales (UID1375),124,0.0,2428396,...,95,1,Bacteria,Firmicutes_A,Clostridia,Clostridiales,Clostridiaceae,Clostridium_P,Clostridium_P perfringens,GCA_000013845
2,GCA_000016465.1,GB_GCA_000016465.1,14,99.55,0.23,767,f__Pasteurellaceae (UID4932),440,0.0,1584718,...,58,1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Pasteurellaceae,Haemophilus,Haemophilus influenzae,GCA_000016465
3,GCA_000024525.1,GB_GCA_000024525.1,0,100.00,0.89,454,o__Cytophagales (UID2936),336,25.0,7481814,...,47,0,Bacteria,Bacteroidota,Bacteroidia,Cytophagales,Spirosomaceae,Spirosoma,Spirosoma linguale,GCA_000024525
4,GCA_000152525.1,GB_GCA_000152525.1,0,96.45,0.11,813,o__Pseudomonadales (UID4488),308,0.0,5429460,...,37,1,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas aeruginosa,GCA_000152525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311475,GCA_910574395.1,RS_GCF_910574395.1,1,99.22,0.78,449,g__Lactobacillus (UID380),129,0.0,1773962,...,56,0,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus,Lactobacillus sp910589675,GCA_910574395
311476,GCA_910589955.1,RS_GCF_910589955.1,0,99.81,0.66,773,g__Staphylococcus (UID294),178,0.0,2102381,...,54,0,Bacteria,Firmicutes,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus epidermidis,GCA_910589955
311477,GCA_910590315.1,RS_GCF_910590315.1,0,99.40,0.00,773,g__Staphylococcus (UID294),178,0.0,2058841,...,59,0,Bacteria,Firmicutes,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus epidermidis,GCA_910590315
311478,GCA_910592735.1,RS_GCF_910592735.1,0,99.97,0.67,1173,f__Enterobacteriaceae (UID5124),336,0.0,4666456,...,81,1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli,GCA_910592735


In [5]:
# RAST
# the full data set. Don't try this at home!
# metadf = pd.read_csv("../small_data/patric_genome_metadata.tsv.gz", compression='gzip', header=0, delimiter="\t")
rast = read_metadata()
rast['assembly_nover'] = rast['assembly_accession'].apply(remove_ver)
rast[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000003135.1,GCA_000003135
1,GCA_000003215.1,GCA_000003215
2,GCA_000003645.1,GCA_000003645
3,GCA_000003925.1,GCA_000003925
4,GCA_000003955.1,GCA_000003955
...,...,...
320171,GCF_900167595.1,GCF_900167595
320172,GCF_900167605.1,GCF_900167605
320173,GCF_900167615.1,GCF_900167615
320174,GCF_900167625.1,GCF_900167625


In [6]:
rast

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,temperature_range,optimal_temperature,salinity,oxygen_requirement,habitat,disease,comments,additional_metadata,isolation_date,assembly_nover
0,GCA_000003135.1,5.484803e+05,Bifidobacterium longum subsp. longum ATCC 55813,Bifidobacterium longum subsp. longum ATCC 55813,548480,WGS,ATCC 55813,NaN,NaN,NaN,...,Mesophilic,-,NaN,Anaerobic,Host-associated,NaN,Bifidobacterium longum subsp. longum ATCC 5581...,NaN,NaN,GCA_000003135
1,GCA_000003215.1,4.556313e+05,Clostridium difficile QCD-66c26,Clostridium difficile QCD-66c26,455631,WGS,QCD-66c26,NaN,NaN,NaN,...,Mesophilic,37,NaN,Anaerobic,Multiple,Antibiotic-associated Diarrhea;Pseudomembraneo...,Clostridium difficile QCD-66c26. was isolated ...,NaN,NaN,GCA_000003215
2,GCA_000003645.1,5.269733e+05,Bacillus cereus m1293,Bacillus cereus m1293,526973,WGS,m1293,NaN,NaN,NaN,...,Mesophilic,-,NaN,Aerobic,Multiple,Food poisoning,"Bacillus cereus m1293. This strain, also know...",NaN,NaN,GCA_000003645
3,GCA_000003925.1,5.269973e+05,Bacillus mycoides DSM 2048,Bacillus mycoides DSM 2048,526997,WGS,DSM 2048,NaN,NaN,NaN,...,NaN,-,NaN,NaN,NaN,NaN,Bacillus mycoides DSM 2048. Bacillus mycoides...,NaN,NaN,GCA_000003925
4,GCA_000003955.1,5.269943e+05,Bacillus cereus AH1273,Bacillus cereus AH1273,526994,WGS,AH1273,NaN,NaN,NaN,...,Mesophilic,-,NaN,Aerobic,Multiple,Food poisoning,Bacillus cereus AH1273. This strain was isola...,NaN,NaN,GCA_000003955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320171,GCF_900167595.1,1.496160e+03,Clostridioides difficile strain VRECD0186,NaN,1496,WGS,VRECD0186,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,These data are part of a pre-publication relea...,NaN,2015.488706,GCF_900167595
320172,GCF_900167605.1,1.147123e+06,Caloramator quimbayensis strain USBA 833,NaN,1147123,WGS,USBA 833,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Caloramator quimbayensis USBA 833,NaN,NaN,GCF_900167605
320173,GCF_900167615.1,1.496159e+03,Clostridioides difficile strain CD19,NaN,1496,WGS,CD19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.sanger.ac.uk/resources/downloads/ba...,NaN,2014.489391,GCF_900167615
320174,GCF_900167625.1,1.496159e+03,Clostridioides difficile strain VRECD0076,NaN,1496,WGS,VRECD0076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,These data are part of a pre-publication relea...,NaN,2014.489391,GCF_900167625


# GenBank

This assembly summary comes from [GenBank ftp site](ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/) and you want the [assembly_summary.txt](ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt) file explicitly from bacteria (but don't try and open the bacteria list in your browser!)

In [7]:
# GenBank
gbk = read_gbk_metadata()
gbk['assembly_nover'] = gbk['assembly_accession'].apply(remove_ver)
gbk[['assembly_accession', 'assembly_nover']]

c:\workspace\edwa0468\githubs\phispyanalysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


,assembly_accession,assembly_nover
0,GCA_019958295.1,GCA_019958295
1,GCA_900128725.1,GCA_900128725
2,GCA_008244535.1,GCA_008244535
3,GCA_008015785.1,GCA_008015785
4,GCA_001735525.1,GCA_001735525
...,...,...
1195588,GCA_914768085.1,GCA_914768085
1195589,GCA_910595835.1,GCA_910595835
1195590,GCA_910595855.1,GCA_910595855
1195591,GCA_910595815.1,GCA_910595815


In [8]:
gbk

,assembly_accession,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,...,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_nover
0,GCA_019958295.1,GCA_019958295.1,PRJNA755163,SAMN20395209,JAIOWO000000000.1,na,9,9,Buchnera aphidicola,strain=Tabriz.4,...,2021/09/13,ASM1995829v1,"Tabriz University, Tabriz, Iran",na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,fragmented assembly,NaN,na,GCA_019958295
1,GCA_900128725.1,GCA_900128725.1,PRJEB15400,SAMEA4556317,NaN,na,9,9,Buchnera aphidicola,strain=BCifornacula,...,2016/11/25,BCifornacula_v1.0,Centre de Biologie pour la Gestion des Populat...,GCF_900128725.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...,NaN,NaN,na,GCA_900128725
2,GCA_008244535.1,GCA_008244535.1,PRJNA562146,SAMN12634886,VTFL00000000.1,na,14,14,Dictyoglomus thermophilum,strain=PYS_80_B,...,2019/09/02,ASM824453v1,Agharkar Research Institute,GCF_008244535.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na,GCA_008244535
3,GCA_008015785.1,GCA_008015785.1,PRJNA428383,SAMN10026444,SSGG00000000.1,na,17,17,Methylophilus methylotrophus,NaN,...,2019/08/19,ASM801578v1,Colorado School of Mines,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,derived from metagenome,NaN,na,GCA_008015785
4,GCA_001735525.1,GCA_001735525.1,PRJNA328548,SAMN05384437,MCBT00000000.1,representative genome,23,23,Shewanella colwelliana,strain=CSB03KR,...,2016/09/19,ASM173552v1,Chonnam National University,GCF_001735525.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,NaN,NaN,na,GCA_001735525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195588,GCA_914768085.1,GCA_914768085.1,PRJEB47378,SAMEA9570072,CAJZCJ000000000.1,na,2946699,2946699,Snodgrassella communis,strain=R-54236,...,2022/02/17,R-54236,Sciensano,GCF_914768085.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...,NaN,NaN,na,GCA_914768085
1195589,GCA_910595835.1,GCA_910595835.1,PRJEB42974,SAMEA8541143,CAJVCI000000000.1,na,2946838,2946838,Candidatus Thioglobus vadi,NaN,...,2021/07/07,SUP05-3,MAX PLANCK INSTITUTE FOR MARINE MICROBIOLOGY,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...,derived from environmental source,assembly from type material,na,GCA_910595835
1195590,GCA_910595855.1,GCA_910595855.1,PRJEB42974,SAMEA8541145,CAJVCJ000000000.1,na,2946839,2946839,Candidatus Thioglobus vulcanius,NaN,...,2021/07/07,SUP05-5,MAX PLANCK INSTITUTE FOR MARINE MICROBIOLOGY,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...,derived from environmental source,assembly from type material,na,GCA_910595855
1195591,GCA_910595815.1,GCA_910595815.1,PRJEB42974,SAMEA8541147,CAJVCM000000000.1,na,2946840,2946840,Candidatus Thioglobus plumae,NaN,...,2021/07/07,SUP05-6,MAX PLANCK INSTITUTE FOR MARINE MICROBIOLOGY,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/9...,derived from environmental source,assembly from type material,na,GCA_910595815


# Phages

In [9]:
phagesdf = read_phages(maxcontigs=-1) # this disables contig length filtering
phagesdf['assembly_nover'] = phagesdf['assembly_accession'].apply(remove_ver)
phagesdf

Please note that this was run with git commit 93794ca that has 1,195,494 genomes parsed.
Initially there were 5,005,011 kept phages, but after filtering we kept 5,005,011 prophages from 945,685 genomes

,assembly_accession,GENOMEID,Total_bp,Contigs,Contigs_Checked,Total_Predicted_Prophages,Kept,No_phage_genes,Not_enough_genes,bp_of_kept_prophages,bp_no_phage_genes,bp_not_enough_genes,Note,assembly_nover
2,GCA_008244535.1,GCA_008244535.1,1963348,23,14,17,4,5,8,79796,104676,13431,Complete,GCA_008244535
3,GCA_008015785.1,GCA_008015785.1,1188656,94,94,35,0,22,13,0,183294,26642,Complete,GCA_008015785
4,GCA_001735525.1,GCA_001735525.1,4642543,52,33,30,3,4,23,83559,58207,33920,Complete,GCA_001735525
5,GCA_019654935.1,GCA_019654935.1,4631520,171,47,13,0,2,11,0,59552,10210,Complete,GCA_019654935
6,GCA_019655395.1,GCA_019655395.1,4473905,472,218,11,0,3,8,0,17577,24469,Complete,GCA_019655395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195489,GCA_020319415.1,GCA_020319415.1,5559579,310,143,76,11,2,63,178853,9738,127058,Complete,GCA_020319415
1195490,GCA_020319515.1,GCA_020319515.1,5497958,201,116,69,13,1,55,215550,5011,105851,Complete,GCA_020319515
1195491,GCA_020319535.1,GCA_020319535.1,5294704,195,95,68,9,0,59,114216,0,133228,Complete,GCA_020319535
1195492,GCA_020319715.1,GCA_020319715.1,4928082,165,87,44,4,0,40,116732,0,73535,Complete,GCA_020319715


# What is in common between the groups?

In [10]:
gbkaa=set(gbk['assembly_accession'])
rastaa=set(rast['assembly_accession'])
gtdbaa=set(gtdb['assembly_accession'])
phagesaa=set(phagesdf['assembly_accession'])


gbr = gbkaa.intersection(rastaa)
gbg = gbkaa.intersection(gtdbaa)
gtr = gtdbaa.intersection(rastaa)

print(f"There are {len(gbkaa):,} genomes in GenBank")
print(f"There are {len(rastaa):,} genomes in PATRIC")
print(f"There are {len(gtdbaa):,} genomes in GTDB")
print(f"We have analyzed phages in {len(phagesaa):,} genomes")
print()
print(f"Between GBK and RAST there are {len(gbr):,} genomes in common")
print(f"Between GBK and GTDB there are {len(gbg):,} genomes in common")
print(f"Between GTDB and RAST there are {len(gtr):,} genomes in common")
print()

gbnotdone = gbkaa - phagesaa
gtnotdone = gtdbaa - phagesaa
ranotdone = rastaa - phagesaa
print(f"There are {len(gbnotdone):,} phages in Genbank that have not been analyzed")
print(f"There are {len(gtnotdone):,} phages in GTDB that have not been analyzed")
print(f"There are {len(ranotdone):,} phages in Genbank that have not been analyzed")
print()

allmissing = gbnotdone.intersection(gtnotdone).intersection(ranotdone)
print(f"There are {len(allmissing):,} phages in all three that have not been analyzed")

if False:
    with open("../data/unprocessed_phages.txt", 'w') as out:
        for o in allmissing:
            gbk[gbk['assembly_accession'] == 'GCA_002129805.1'][['assembly_accession', 'ftp_path']].to_csv(out, sep="\t", header=False, index=False)

There are 1,195,593 genomes in GenBank
There are 320,176 genomes in PATRIC
There are 311,468 genomes in GTDB
We have analyzed phages in 945,685 genomes

Between GBK and RAST there are 242,239 genomes in common
Between GBK and GTDB there are 308,578 genomes in common
Between GTDB and RAST there are 205,940 genomes in common

There are 249,908 phages in Genbank that have not been analyzed
There are 115,918 phages in GTDB that have not been analyzed
There are 176,920 phages in Genbank that have not been analyzed

There are 83,931 phages in all three that have not been analyzed


In [11]:
print("IF WE IGNORE GENOME VERSIONS")
gbkaa=set(gbk['assembly_nover'])
rastaa=set(rast['assembly_nover'])
gtdbaa=set(gtdb['assembly_nover'])
phagesaa=set(phagesdf['assembly_nover'])


gbr = gbkaa.intersection(rastaa)
gbg = gbkaa.intersection(gtdbaa)
gtr = gtdbaa.intersection(rastaa)

print(f"There are {len(gbkaa):,} genomes in GenBank")
print(f"There are {len(rastaa):,} genomes in PATRIC")
print(f"There are {len(gtdbaa):,} genomes in GTDB")
print(f"We have analyzed phages in {len(phagesaa):,} genomes")
print()
print(f"Between GenBank and PATRIC there are {len(gbr):,} genomes in common")
print(f"Between GenBank and GTDB there are {len(gbg):,} genomes in common")
print(f"Between GTDB and PATRIC there are {len(gtr):,} genomes in common")
print()

gbnotdone = gbkaa - phagesaa
gtnotdone = gtdbaa - phagesaa
ranotdone = rastaa - phagesaa
print(f"There are {len(gbnotdone):,} phages in Genbank that have not been analyzed")
print(f"There are {len(gtnotdone):,} phages in GTDB that have not been analyzed")
print(f"There are {len(ranotdone):,} phages in PATRIC that have not been analyzed")
print()

allmissing = gbnotdone.intersection(gtnotdone).intersection(ranotdone)
print(f"There are {len(allmissing):,} phages in all three that have not been analyzed")

IF WE IGNORE GENOME VERSIONS
There are 1,195,593 genomes in GenBank
There are 320,033 genomes in PATRIC
There are 311,468 genomes in GTDB
We have analyzed phages in 945,685 genomes

Between GenBank and PATRIC there are 247,087 genomes in common
Between GenBank and GTDB there are 309,616 genomes in common
Between GTDB and PATRIC there are 209,749 genomes in common

There are 249,908 phages in Genbank that have not been analyzed
There are 115,149 phages in GTDB that have not been analyzed
There are 173,347 phages in PATRIC that have not been analyzed

There are 85,249 phages in all three that have not been analyzed
